In [1]:
#Load games

import nflgame
games = nflgame.games(2015, week=1)

In [3]:
# Extract features
import re
from collections import defaultdict

play_features = []

for p in nflgame.combine_plays(games).limit(100000):
    features = defaultdict(float)
    if (p.note == None or p.note== 'TD' or p.note=='INT') and ('punt' not in p.desc) and ('END ' not in p.desc) and ('End ' not in p.desc) \
    and ('Two-Minute Warning' not in p.desc) and ('spiked the ball to stop the clock' not in p.desc) \
    and ('Delay of Game' not in p.desc):
        #print "Description:",p.desc
        
        features['team'] = p.team        
        features['time'] = p.time.clock
        features['quarter'] = p.time.qtr            
        features['position'] = 50-p.yardline.offset
        features['down'] = p.down
        features['togo'] = p.yards_togo
        
        if 'Shotgun' in p.desc:
                    features['shotgun'] = 1
                
        if 'incomplete' in p.desc:
            features['complete'] = 0
            features['pass'] = 1
            rematch = re.search(r'incomplete \S+ \S+ to', p.desc)
            
            if rematch is None:
                # ball just thrown away, no intended target -> ignore
                continue;
                
            match = rematch.group(0).split()            
            features['dist'] = match[1]
            features['side'] = match[2]
        else:
            if 'no gain' in p.desc:
                features['yards'] = 0
            else:
                if p.note=='INT':
                    features['intercepted'] = 1 
                else:                
                    rematch = re.search(r'[-]?[0-9]+ yard\s?', p.desc) 
                    if rematch is None:
                        print p.note
                        print p.desc
                    match = rematch.group(0)         
                    features['yards'] = float(match[:match.find(' ')])
            
            if 'pass' in p.desc:
                features['pass'] = 1
                match = re.search(r'pass \S+ \S+', p.desc).group(0).split()
                features['dist'] = match[1]
                features['side'] = match[2]
                
                if p.note!='INT':
                    features['complete'] = 1 
            else:
                features['pass'] = 0
                if 'up to the middle' in p.desc:
                    features['side'] = 'middle'
                else:
                    rematch = re.search(r'\S+\.\S+ \S+ \S+', p.desc)                    
                    match = rematch.group(0).split()
                    features['side'] = match[1] + ' ' + match[2]
                
            if features['yards'] >= p.yards_togo:
                features['firstdown'] = 1
            if p.touchdown == True:
                features['touchdown'] = 1
        #print features
        play_features.append(features)
    #else:
    #    print ">> Ignore:",p.note,'-> '+ p.desc
print len(play_features)
        

1962
